In [7]:
from transformers import BertForQuestionAnswering, BertTokenizer, AdamW
from datasets import load_dataset
import torch

# Carica il modello preaddestrato e il tokenizzatore di BERT
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
dataset=load_dataset('squad',split='train')


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
dataset=dataset[:5]
answer_starts=[item['answer_start']for item in dataset['answers']]
print(answer_starts)

[[515], [188], [279], [381], [92]]


In [9]:
# Dataset di esempio per il fine-tuning (da sostituire con il tuo dataset)
train_contexts = dataset['context'] # Lista dei contesti
train_questions = dataset['question']  # Lista delle domande
train_start_positions = answer_starts  # Lista degli indici di inizio delle risposte

# Tokenizzazione e preparazione dei dati
inputs = tokenizer(train_questions, train_contexts, return_tensors='pt', padding=True, truncation=True)
start_positions = torch.tensor(train_start_positions)

# Parametri di addestramento
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
batch_size = 8

# Addestramento del modello
for epoch in range(num_epochs):
    for i in range(0, len(train_questions), batch_size):
        optimizer.zero_grad()
        batch_inputs = {key: val[i:i+batch_size] for key, val in inputs.items()}
        outputs = model(**batch_inputs, start_positions=start_positions[i:i+batch_size])
        print(outputs)




# Salvataggio del modello dopo il fine-tuning
model.save('fine_tuned_bert_model')

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ -6.0380,  -6.3095,  -6.6209,  -8.1536,  -8.4597,  -8.2306,  -8.6416,
          -8.2747,  -6.7943,  -8.6180,  -8.9770,  -8.3109,  -7.4786,  -8.7998,
          -8.4728,  -9.3864,  -6.0380,  -7.6236,  -8.8612,  -8.8072,  -7.7707,
          -7.4811,  -8.6336,  -8.3924,  -7.4226,  -8.2635,  -8.7095,  -6.5404,
          -8.6135,  -8.1074,  -8.0168,  -7.7479,  -8.8593,  -7.6560,  -8.1699,
          -8.2829,  -7.5939,  -7.0934,  -6.6524,  -8.2448,  -7.1872,  -5.9503,
          -5.2114,  -8.3499,  -7.4930,  -8.5293,  -8.2416,  -8.9366,  -8.7302,
          -8.5435,  -8.7286,  -8.9979,  -8.0853,  -8.8754,  -8.8475,  -8.0950,
          -7.7933,  -7.3410,  -7.3273,  -8.6411,  -6.3635,  -8.7465,  -8.3764,
          -8.7561,  -8.6998,  -9.2505,  -8.7506,  -8.4835,  -8.5575,  -8.4158,
          -7.8081,  -8.7068,  -9.0581,  -8.7142,  -8.9947,  -8.6764,  -9.1367,
          -9.1498,  -7.9129,  -6.5686,  -8.6370,  -8.3253,  -8.0899,  -8.3979,

AttributeError: 'BertForQuestionAnswering' object has no attribute 'save'

In [ ]:
# Esempio di domanda e contesto per il question answering
question = dataset['question'][2]
context = dataset['context'][2]
answers = dataset['answers'][2]
print(answers)



input_ids = tokenizer.encode(question, context)
attention_mask = [1] * len(input_ids)
output = model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))
start_index = torch.argmax(output[0][0, :len(input_ids) - input_ids.index(tokenizer.sep_token_id)])
end_index = torch.argmax(output[1][0, :len(input_ids) - input_ids.index(tokenizer.sep_token_id)])
answer = tokenizer.decode(input_ids[start_index:end_index + 1], skip_special_tokens=True)
print("Answer:", answer)

{'text': ['the Main Building'], 'answer_start': [279]}
Answer: the main building
